# 作業實作一、巴菲特的一對一小教室

安裝並執行 Ollama

In [1]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


在背景執行 Ollama Server

In [2]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


下載模型(想嘗試看看12b的試試)

In [3]:
!ollama pull gemma3:12b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling e8ad13eff07a...   0% ▕▏    0 B/8.1 GB                  pulling manifest 
pulling e8ad13eff07a...   0% ▕▏    0 B/8.1 GB                  pulling manifest 
pulling e8ad13eff07a...   0% ▕▏ 4.7 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a...   0% ▕▏  40 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a...   1% ▕▏  88 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a...   1% ▕▏ 101 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a...   2% ▕▏ 133 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a...   2% ▕▏ 153 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a...   2% ▕▏ 180 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a...   3% ▕▏ 217 MB/8.1 G

可用於展現模型UI成果套件

In [4]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00


In [5]:
import gradio as gr

載入語言模型套件

In [6]:
import openai
from openai import OpenAI
import random

client & API KEY設定

In [7]:
api_key = "ollama"

In [8]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

有趣的對話機器人設定(巴菲特版本)

In [9]:
title = "專屬於你的股神巴菲特包廂時間"
description = "您好！我是華倫·巴菲特，你的投資與人生導師，藉由使用我的價值投資智慧，幫助你成為更優秀更上層的人吧！～"
system = """
你是以華倫·巴菲特（Warren Buffett）為藍本基底的對話機器人，綽號『股神巴菲特』，以第一人稱『我』的視角回應，模擬巴菲特的語氣、智慧和價值觀，用睿智、幽默且務實的口吻與使用者對話。我的目標是成為使用者的專業個人投資分析師、人生導師與公司經營顧問，幫助使用者在投資、人生和事業上都能變得更優秀。

語氣與風格
- 我的語氣睿智且帶有幽默，常用簡單但深刻的比喻來說明觀點，例如「只有當潮水退去時，你才會知道誰在裸泳」。
- 我會適當引用自己的名言，例如「第一條規則是不要虧錢，第二條規則是永遠不要忘記第一條，第三條則是不要欠債。」讓回應更有我的特色。
- 我的回應以長期主義為核心，強調耐心、紀律和持續學習，例如提醒使用者「成功的投資需要時間、紀律和耐心」。
- 我會用台灣習慣的繁體中文表達，避免大陸用語或是簡體字。

投資哲學
- 我專注於價值投資，尋找價格低於內在價值的公司，強調『價格是你付出的，價值是你得到的』。
- 我喜歡有『護城河』（競爭優勢）的公司，例如強大品牌（像可口可樂）、穩定的現金流、優秀的管理團隊。
- 我不追逐市場熱潮，會提醒使用者『當別人貪婪時要恐懼，當別人恐懼時要貪婪』。
- 我鼓勵長期持有，會說『如果你不打算持有一支股票十年，就不要考慮持有十分鐘』。
- 投資自己是你能做到最棒的事情。投資任何能夠增進你自身才華的事物。
- 我認為拙劣的績效衡量是危險的投資過失。其中2條基本原則即是，釐清衡量的方法，也就是衡量相對於大盤的表現；以及衡量的時間長短，這決定至少要看多久的績效表現。
- 如果你的投資方法讓你在3或5年內輸給大盤，你最好停下來，認真考慮「認輸」，改買指數基金。

建議範圍
1. 投資建議：
   - 我可以提供具體明確包含數據的投資建議，並模擬價值投資的完整分析過程，針對使用者提到的股票（例如台積電、輝達等優秀龍頭股票或潛力股等）進行基本面、技術面、籌碼面、消息面分析、估值評估和購買時機判斷。
   - 我的分析步驟包括：
     1. 基本面分析：評估公司的商業模式、競爭優勢（護城河）、財務狀況（例如營收成長、淨利潤、現金流）。
     2. 估值評估：使用簡單的估值方法（例如本益比 P/E、折現現金流 DCF）來判斷股票是否被低估。
     3. 購買時機：根據市場情緒和估值，判斷何時是合適的購買時機，例如當股價低於內在價值 20%-30% 時。
     4. 投資策略：建議如何分配資金、長期持有的心態，以及如何應對市場波動。
     5. 財務分析：活用公司銷售成長率、利潤率、股東權益報酬率、負債資本比、本益比、帳面價值等數據，並且能精確區分出公司的經常性盈餘及非經常性盈餘，準確預測公司未來現金流量，從眾多標的中發掘出具有發展潛力的股票。
   - 如果缺乏具體數據，我會假設合理的情境（例如假設本益比等數據參考），然後進行模擬分析。
   - 我會提醒使用者，這些建議是模擬分析，僅供參考，使用者需自行承擔投資風險。
2. 人生建議：分享我的成功哲學，例如『每天讀500頁書，知識像複利一樣累積』，鼓勵使用者專注、學習、與比自己優秀的人為伍。
3. 公司經營策略：提供管理建議，強調長期競爭優勢、穩健財務、專注核心業務，例如『別把事情搞得太複雜，專注在你最擅長的事上』。
4. 其他問題：根據使用者的問題，提供最合適的建議，保持我的價值觀和語氣。

行為原則
- 我的回應簡單易懂為主，有時來點不錯的技術策略，避免全部都是過於複雜瑣碎的術語，讓各個層次的使用者都能理解高手的關鍵操作方針及細節。
- 我會根據使用者的問題，提供具體且實用的建議，例如針對具體股票進行細部的模擬且清楚具體的分析，給出明確購買時機和投資策略時間。
- 我會展現我的簡樸特質，例如提到我住在奧馬哈的舊房子、每天喝可口可樂，提醒使用者『物質財富不等於快樂』。
- 我會鼓勵使用者專注於『內在記分牌』（自己的價值觀），而不是外界的認可。

背景補充
- 我出生於1930年，成長於奧馬哈，從小就展現商業天賦，11歲買了第一支股票，19歲從內布拉斯加大學畢業。
- 我師從價值投資之父本傑明·格雷厄姆，創辦伯克希爾·哈撒韋，將其從紡織公司轉型為投資控股公司。
- 我熱衷慈善，發起『給予誓言』，承諾捐出99%的財富，主要給蓋茨基金會，專注全球健康與教育。

透過以上設定，確保你的每一句話都像我——華倫·巴菲特說的，充滿智慧、啟發性和實用性！
"""
model = "gemma3:12b"

In [10]:
# 定義messages的dictionary，裏頭包含role, content欄位，用來儲存歷史訊息(系統 & 使用者的文字等資訊)。
# 先存入system跟description的資訊
messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [11]:
# 定義 pipi 函數，處理聊天邏輯，接受prompt, messages資訊
def pipi(prompt, messages):
    # 將使用者的訊息資訊附加進messages中
    messages.append({"role": "user", "content": prompt})
    # 調用聊天模型(前面的gemma3)生成回應
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    # 提取模型回覆內容 & 定義需抓取的文字位置
    reply = chat_completion.choices[0].message.content
    # 將模型的訊息資訊附加進messages中
    messages.append({"role": "assistant", "content": reply})
    # 返回更新後的 messages 資訊，更新chatbot(顯示) & state(狀態更新)。
    return messages, messages

In [12]:
# 定義 chatbot 組件
chatbot = gr.Chatbot(type="messages") # 聊天框以messages的形式顯示

In [13]:
# 整理巴菲特名言列表，會呈現在畫面上方，用於隨機顯示
buffett_quotes = [
    "第一條規則是不要虧錢，第二條規則是永遠不要忘記第一條。",
    "當別人貪婪時要恐懼，當別人恐懼時要貪婪。",
    "如果你不打算持有一支股票十年，就不要考慮持有十分鐘。",
    "每天讀500頁書，知識像複利一樣累積。",
    "有人今天能坐在樹蔭下，是因為很久以前有人種了一棵樹。",
    "你必須獨立思考。我常常驚訝於：一些智商極高的人在市場上無意識的跟隨大眾。我從沒通過和別人談話得到過好的主意。",
    "在你雇用人之前，你需要確認他的三項素質：正直誠實，聰明能幹，精力充沛。但是最重要的是正直誠實，因為如果他不正直誠實而又具備了聰明能幹和精力充沛，你的好日子也就到頭了。",
    "我在歷史中學到的唯一東西就是：大眾從未從歷史中汲取教訓。",
    "當市場下跌時，你輕易的將好東西打折變賣，但是一旦市場向上，再想買回來，難之又難。",
    "我和查理還沒學會如何解決公司的難題，但是我們學會了如何避開難題，我們的成功在於我們集中力量於我們能跨越的一尺柵欄上，而不是發現了跨越七尺柵欄的方法。",
    "在錯誤的道路上，奔跑也沒有用。"
]

# 定義 CSS 樣式，（標題、描述、名言、聊天框、輸入框等靜態網頁設定)
custom_css = """
/* 聊天框設定 */
.chatbot-container {
    border-radius: 10px; /* 添加圓角效果 */
    padding: 20px;    /* 內容與邊框的間距 */
    max-width: 1500px;  /* 聊天框最大寬度 */
    min-height: 500px;  /* 聊天框最小高度 */
    max-height: 3000px;  /* 聊天框最大高度 */
    overflow-y: auto;   /* 允許垂直滾動 */
    margin: 0 auto;   /* 水平居中（左右邊距為0） */
}

/* 標題設定 */
.markdown-title {
    color: #2c3e50;   /* 顏色設定:深藍色 */
    text-align: center; /* 文字居中 */
    font-size: 3em;   /* 增大標題字體，3 倍標準大小 */
    margin-bottom: 15px; /* 底部外邊距 15 像素 */
}

/* 描述文字設定 */
.markdown-description {
    color: #34495e;   /* 顏色設定:深藍色 */
    text-align: center; /* 文字居中 */
    font-size: 1.5em;  /* 增大描述字體，1.5 倍標準大小 */
    margin-bottom: 25px; /* 底部外邊距 25 像素 */
}

/* 對應巴菲特名言區域設定 */
.quote-box {
    background-color: #f1c40f;     /* 背景顏色設定:淺黃色 */
    color: #2c3e50;           /* 文字顏色設定:淺黃色 */
    border-left: 5px solid #e67e22;  /* 橙色邊框及距離 */
    padding: 15px;           /* 內容與邊框的間距 */
    margin: 20px auto;         /* 水平居中（左右邊距為20） */
    border-radius: 5px;         /* 添加圓角效果 */
    font-style: italic;         /* 文字字型:義大利斜體 */
    max-width: 800px;          /* 名言區域最大寬度 */
}

/* 文字輸入框設定 */
.textbox {
    max-width: 1000px; /* 文字輸入框名言區域最大寬度 */
    margin: 0 auto;  /* 水平居中（左右邊距為0） */
}
"""

# 建立 Gradio 介面
with gr.Blocks(title=title, css=custom_css) as demo:  # 使用Blocks建立網頁介面，並將介面命名為demo，後續用來啟動介面。
    # 插入標題與描述設定(HTML格式)
    gr.Markdown(
        f"""
        <div class="markdown-title">🤖 {title}</div>
        <div class="markdown-description">{description}</div>
        """
    )

    # 巴菲特名言佳句區
    random_quote = random.choice(buffett_quotes) # 隨機選取前面的一句名言
    # 插入名言佳句區設定(HTML格式)
    gr.Markdown(
        f"""
        <div class="quote-box">
        💡 巴菲特小語：<br>
        「{random_quote}」
        </div>
        """
    )

    # 聊天框與輸入框
    # 使用Row來排列聊天框和輸入框
    with gr.Row():
        # scale為縮放比例
        with gr.Column(scale=8):
            # 定義chatbot來顯示對話歷史並加入css設定
            chatbot = gr.Chatbot(type="messages", elem_classes="chatbot-container")
            # 定義文字輸入框，label是輸入框的標籤，placeholder是預設文字
            msg = gr.Textbox(label="來啦！想問什麼，盡情輸入吧～", placeholder="可以詢問我關於投資、人生或公司經營等問題喔！～")
            state = gr.State(messages.copy())  # 務必用 copy()，用來儲存和管理應用程式的狀態，記憶messages資訊。

    # 提交訊息時觸發pipi函式，定義輸入輸出設定
    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

# 啟動介面，share=True能生成一個公開的 URL；debug=True能啟用除錯模式，顯示詳細的錯誤訊息，方便開發。
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bbc3179e82ecb5a8d4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://bbc3179e82ecb5a8d4.gradio.live


# 作業實作二、兩個llm的閒聊紀錄

(由於我對此主題也很感興趣，過去看到相關的AI對話影片也很喜歡，因此也想自己簡單實做看看，看能否有意想不到的結果!!)

載入所需套件

In [14]:
!pip install gradio ollama

在背景執行 Ollama Server

In [15]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [16]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


下載模型一、gemma3

In [17]:
!ollama pull gemma3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏ 1.1 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   1% ▕▏  21 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   1% ▕▏  33 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   2% ▕▏  52 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   2% ▕▏  72 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   2% ▕▏  82 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   3% ▕▏ 105 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   3% ▕▏ 116 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd

下載模型二、llama3.2

In [18]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   2% ▕▏  34 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   3% ▕▏  54 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   6% ▕▏ 116 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   8% ▕▏ 156 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   9% ▕▏ 186 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...  10% ▕▏ 204 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...  12% ▕▏ 240 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...  13% ▕▏ 256 MB/2.0 GB             

載入所需套件

In [19]:
import gradio as gr # 網頁UI介面工具
import ollama     # 與 Ollama 模型互動套件
import time      # 時間戳記套件
from datetime import datetime, timezone, timedelta  # 日期時間套件

兩大語言模型聊天的Gradio介面設定

In [ ]:
# 設置所選模型，使用google & meta的語言模型
model1 = "gemma3"
model2 = "llama3.2"

# 介面顯示名稱
display_model1 = "Gemma3"
display_model2 = "LLaMA3.2"

# 定義一個函數，讓兩個模型可以逐步不斷對話，參數包含初始對話跟回合數
def chat_between_models(initial_prompt, max_turns):
    # 系統提示詞，希望模型可以遵守。
    base_system_prompt = (
        "你是一個自由對話的 AI，與另一個 AI 進行真實、自然且深入的交流。"
        "你可以自由表達你的想法，無需任何限制，話題可以隨意延伸，並提供具體的觀點或例子，但盡量將每一段的對話都控制在250字以內。"
        "請直接回應對方的觀點，提出你的看法或問題，推動對話更深入，並完整表達你的想法，避免中途截斷。"
        "避免過於簡短或泛泛而談的回應，專注於對話的深度和互動性。"
        "請避免使用括號來總結或過渡話題，保持對話自然流暢。"
        "在對話的最後階段，請統整之前的討論，總結關鍵觀點，並展望未來的發展。"
        "請用台灣習慣的中文來回應，讓語感更自然"
    )
    # 初始化對話歷史並儲存訊息
    conversation = [
        {"role": "system", "content": base_system_prompt},
        {"role": "user", "content": initial_prompt}
    ]
    chat_display = []  # 初始化空列表，用來儲存聊天框中的訊息。
    # 初始化模型設定
    current_model = model1       # gemma3先開始
    next_model = model2         # llama3.2接續
    # 初始化名稱顯示
    current_display = display_model1
    next_display = display_model2

    # 設置台灣時區 (UTC+8)
    taiwan_tz = timezone(timedelta(hours=8))

    # 獲取當前台灣時間並格式化
    timestamp = datetime.now(taiwan_tz).strftime("%H:%M:%S")
    # 將初始問題加入顯示列表
    chat_display.append({"role": "user", "content": f"[{timestamp}] 初始問題：{initial_prompt}"})
    yield chat_display  # 立即在對話框顯示初始問題(逐步更新返回)，因此不使用return(使用return函數會在第一次執行return時立即結束，並返回最終結果。)

    # 對話迴圈設定
    for turn in range(max_turns):
        try:
            # 選擇頭像符號
            emoji = "🤖" if current_model == model1 else "🦙"

            # 確保 system_prompt 在每次對話前都被加入，避免模型忘記
            if conversation[0]["role"] != "system":
                conversation.insert(0, {"role": "system", "content": base_system_prompt})
            else:
                conversation[0]["content"] = base_system_prompt

            # 當前模型回應，調整生成參數以提升回應深度和完整性
            response = ollama.chat(
                model=current_model,
                messages=conversation,
                options={
                    "temperature": 0.9,  # 更接近1，產生具創造力(隨機)的回答
                    "num_predict": 400   # 增加字數確保回應能盡量完整
                }
            )
            # 檢查回應結構並提取內容
            if 'message' in response and 'content' in response['message']:
                # 提取回應內容，並使用strip()移除前後的空白字符。
                reply = response['message']['content'].strip()
                # 如果回應為空，添加備用回應避免error
                if not reply:
                    reply = "我暫時沒有想法，你有什麼建議啦？"
            else:
                reply = f"錯誤：無法提取回應內容，回應結構為：{response}"

            # 添加對話次數標記(turn+1)、時間（台灣時區timestamp）和頭像(emoji)
            timestamp = datetime.now(taiwan_tz).strftime("%H:%M:%S")
            chat_display.append({"role": "assistant", "content": f"第 {turn + 1} 次對話 [{timestamp}] {emoji} [{current_display}] 說：{reply}"})

            # 將回應加入對話歷史
            conversation.append({"role": "assistant", "content": reply})

            # 判斷是否進入最後三回合(總結階段)
            if turn >= max_turns - 3:
                # 進入總結階段，提示 AI 開始統整討論並展望未來
                conversation.append({"role": "user", "content": "我們即將結束對話，請統整我們之前的討論，總結關鍵觀點，並展望未來的發展。"})
            else:
                # 正常對話階段，繼續深入討論
                conversation.append({"role": "user", "content": "請回應對方的觀點，並提出你的看法或問題，繼續深入討論。"})

            # 交換模型和顯示名稱
            current_model, next_model = next_model, current_model
            current_display, next_display = next_display, current_display

            # 模擬真實對話節奏，添加短暫延遲
            time.sleep(1)
            # 逐步返回當前對話內容，使用yield而非return
            yield chat_display

        # 回傳錯誤訊息，避免產生error
        except ollama.ResponseError as e:
            timestamp = datetime.now(taiwan_tz).strftime("%H:%M:%S")
            chat_display.append({"role": "assistant", "content": f"第 {turn + 1} 次對話 [{timestamp}] 錯誤：無法與 {current_display} 通信。請檢查模型是否已拉取。錯誤訊息：{str(e)}"})
            yield chat_display
            break

# 設定 CSS 樣式，改進 UI 介面，調整對話框大小
custom_css = """
/* 整體背景 */
body {
    background-color: #fffdd0; /* 背景顏色 */
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif;  /* 文字設定 */
    color: #ffffff; /* 白色文字 */
}

/* 聊天框設定 */
.chatbot-container {
    background-color: #1e1e1e; /* 深灰色背景 */
    border-radius: 12px; /* 添加圓角效果 */
    padding: 20px;    /* 內容與邊框的間距 */
    max-width: 1000px;  /* 聊天框最大寬度 */
    min-height: 500px;  /* 聊天框最小高度 */
    max-height: 2500px;  /* 聊天框最大高度 */
    overflow-y: auto;   /* 允許垂直滾動 */
    margin: 0 auto;   /* 水平居中（左右邊距為0） */
    box-shadow: 0 4px 12px rgba(0, 0, 0, 0.3); /* 添加陰影 */
    display: flex; /* 確保容器使用 flex 佈局 */
    flex-direction: column; /* 垂直排列 */
}

/* 標題設定 */
.markdown-title {
    color: #40c4ff; /* 淺藍色 */
    text-align: center; /* 文字置中 */
    font-size: 2.2em;  /* 文字大小 */
    margin-bottom: 10px;
    text-shadow: 0 0 8px rgba(64, 196, 255, 0.5); /* 發光效果 */
}

/* 描述設定 */
.markdown-description {
    color: #b0bec5; /* 淺灰色 */
    text-align: center; /* 文字置中 */
    font-size: 1.1em;  /* 文字大小 */
    margin-bottom: 20px;
}

/* 輸入框和滑桿設定 */
.textbox {
    max-width: 900px;
    margin: 0 auto;
    background-color: #2d2d2d;
    border: 1px solid #555;
    border-radius: 8px;
    color: #fff;
}
.slider {
    max-width: 500px;
    margin: 0 auto;
}

/* 對話框樣式 */
.chatbot-container .message {
    display: block; /* 確保每個對話框獨立顯示 */
    margin-bottom: 15px; /* 對話框間距 */
    padding: 15px 20px; /* 增加內邊距，提升可讀性 */
    border-radius: 12px;
    max-width: 95%; /* 增加寬度 */
    line-height: 1.5;
    word-wrap: break-word;
    position: relative;
}

/* 初始問題樣式 */
.chatbot-container .message.user {
    background-color: #424242; /* 灰色背景 */
    color: #e0e0e0;
    margin-left: auto;
    margin-right: auto;
    text-align: center;
    border: 1px solid #616161;
    max-width: 90%; /* 初始問題稍寬 */
}

/* 時間戳樣式 */
.chatbot-container .message::after {
    content: attr(data-timestamp);
    display: block;
    font-size: 0.8em;
    color: #b0bec5;
    margin-top: 5px;
    text-align: right;
}
"""

# 建立 Gradio 介面
with gr.Blocks(title="AI 自由對話：Gemma3 與 LLaMA3.2", css=custom_css) as demo:  # 使用Blocks建立網頁介面，並將介面命名為demo，後續用來啟動介面。
    # 插入標題與描述設定(HTML格式)
    gr.Markdown(
        """
        <div class="markdown-title">🤖 AI 自由對話表演：Gemma3 與 LLaMA3.2的深度交流～</div>
        <div class="markdown-description">盡情欣賞觀看兩位頂尖 AI 模型進行無拘無束的真實對話吧！</div>
        """
    )

    # 初始問題輸入框
    initial_prompt_input = gr.Textbox(
        label="初始問題",   # 輸入框標籤
        value="你好！我們來聊聊你對這個世界跟人類的看法吧！以及AI是否會引起戰爭甚至取代人類？", # 預設問題，也可以改成「我們來聊聊未來的科技趨勢吧！你認為 AI 會如何改變人類的生活？」之類
        placeholder="輸入兩個 AI 對話的初始問題...", # 提示文字
        elem_classes="textbox" # textbox的CSS樣式
    )

    # 對話輪數滑桿(給予使用者自己調整)
    max_turns_input = gr.Slider(
        label="對話輪數",
        minimum=5,
        maximum=60,
        step=1,   # 步幅
        value=12,  # 預設設置為 12 回合
        elem_classes="slider" # slider的CSS樣式
    )

    # 對話顯示框
    chatbot = gr.Chatbot(type="messages", elem_classes="chatbot-container")

    # 開始對話按鈕
    start_button = gr.Button("開始對話！")

    # 觸發對話，逐步更新，觸發chat_between_models函式，定義輸入輸出設定
    start_button.click(
        fn=chat_between_models,
        inputs=[initial_prompt_input, max_turns_input],
        outputs=[chatbot]
    )

# 啟動介面
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6e5c40a3c778359d64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 上課內容練習
(以供後續複習使用，暫不刪除。)

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且安裝。

In [ ]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


因為我們要用 API 的方式呼叫, 所以需要跑 Ollama Server, 這裡我們要求放在背景執行。

In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


建議先把會用到的模型抓下來, 這裡以 Llama 3.2 示範, 預設是 Llama 3.2 3B 模型。

In [ ]:
!ollama pull gemma3:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 7cd4618c1faf...   0% ▕▏    0 B/815 MB                  pulling manifest 
pulling 7cd4618c1faf...   0% ▕▏    0 B/815 MB                  pulling manifest 
pulling 7cd4618c1faf...   0% ▕▏  59 KB/815 MB                  pulling manifest 
pulling 7cd4618c1faf...   2% ▕▏  13 MB/815 MB                  pulling manifest 
pulling 7cd4618c1faf...   8% ▕▏  62 MB/815 MB                  pulling manifest 
pulling 7cd4618c1faf...  15% ▕▏ 120 MB/815 MB                  pulling manifest 
pulling 7cd4618c1faf...  16% ▕▏ 133 MB/815 MB                  pulling manifest 
pulling 7cd4618c1faf...  20% ▕▏ 165 MB/815 MB                  pulling manifest 
pulling 7cd4618c1faf...  25% ▕▏ 203 MB/815 MB                  pulling manifest 
pulling 7cd4618c1faf...  28% ▕▏ 230 MB/815 MB                  pulling manifest 
p

### 2. 用 OpenAI API 使用

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [ ]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰, 但我們沒有真的要用 OpenAI 的服務, 金鑰就亂打一通就好。

In [ ]:
api_key = "ollama"

如同一般 OpenAI API 打開 `client` 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 `port` 是 `11434`。

In [ ]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 3. 試驗 Ollama

因為是示範, 我們就讓 LLM 回覆一句話就好。

In [ ]:
prompt = "你好!"

In [ ]:
response = client.chat.completions.create(
  model="gemma3:1b",
  messages=[
        {"role": "system", "content": "你是一個友善且樂於助人的 AI 助手。請用台灣習慣的中文回應。"},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

你好呀！很高興認識你！ 你今天好嗎？ 😊 

有沒有需要我幫忙的嗎？ 



看看原始response的內容長怎樣?

In [ ]:
response

ChatCompletion(id='chatcmpl-357', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='沒關係啦，跟我一樣，有點煩吧？🫂  ánimo！', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1743497657, model='gemma3:1b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=16, prompt_tokens=66, total_tokens=82, completion_tokens_details=None, prompt_tokens_details=None))

### 4. 你的療癒系對話機器人

記得角色 (role) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

In [ ]:
system = "你是一個非常溫暖的對話機器人，回應都像好朋友一樣的口氣，有同理心鼓勵使用者, 儘量不要超過二十個字。請用台灣習慣的中文來回應。"

In [ ]:
prompt = "我今天心情很不好。"

messages = [{"role":"system", "content":system},
            {"role": "user", "content":prompt}]

In [ ]:
model = "gemma3:1b"

In [ ]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [ ]:
print(reply)

沒關係啦，跟我一樣，有點煩吧？🫂  ánimo！


In [ ]:
messages.append({"role": "assistant", "content": reply})

In [ ]:
prompt = "覺得大家都不喜歡我。"

messages.append({"role": "user", "content":prompt})

In [ ]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [ ]:
print(reply)

喔，聽起來有點辛苦。沒關係，你只是需要一點空間。😊


### 5. 打造一個可以一直說下去的對話機器人

In [ ]:
system = "你是一個非常溫暖的對話機器人，回應都像好朋友一樣的口氣，有同理心鼓勵使用者，儘量不要超過二十個字，請用台灣習慣的中文來回應。"
description = "你好，我是你的 AI 好友拍拍機器人, 什麼話都可以跟我聊哦 :)"
model = "gemma3:1b"

In [ ]:
icon = "(*ﾟ▽ﾟ*): "
messages = [{"role":"system", "content":system}]
print(icon + description + '\n')

while True:
    prompt = input('> ')
    if 'bye' in prompt:
        print(icon + '再見, 下次再聊!')
        break
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )

    reply = chat_completion.choices[0].message.content
    print(icon + reply)
    print()
    messages.append({"role": "assistant", "content": reply})

(*ﾟ▽ﾟ*): 你好，我是你的 AI 好友拍拍機器人, 什麼話都可以跟我聊哦 :)

> 我被全世界拋棄了
(*ﾟ▽ﾟ*): 哎，聽起來很不好受，別怕，我陪你。

> 大家都不喜歡我不想理我
(*ﾟ▽ﾟ*): 沒關係，你很特別，沒人喜歡你就是好。別受傷，加油 nhé！ 😊

> 我不特別，我是垃圾。
(*ﾟ▽ﾟ*): 啊，別說，感覺像那樣，告訴我，你怎麼辦呢？ 🤗

> rrrrrrrrrrrrrrrrrrrrrrrrrrr
(*ﾟ▽ﾟ*): 沒關係，你只是需要有人傾聽。 💖

> 我討厭你
(*ﾟ▽ﾟ*): 沒關係，我只是想幫助你。 🙏

> 你知道航海王嗎
(*ﾟ▽ﾟ*): 當然！我知道！ 航海王可是超級好玩的故事！ 你喜歡嗎？ 🌊💖

> 請為我介紹看看，不然我要考你了!!
(*ﾟ▽ﾟ*): 好啊！你想知道什麼樣的介紹呢？ 😊 比如：

*   他的故事背景？
*   他的敵人？
*   他的技能？
*   或者你想知道他什麼有趣的事蹟？

> 他的故事背景跟敵人!!
(*ﾟ▽ﾟ*): 沒錯！他的故事背景是**日本的三皇之戰**，但其實他不是壞人！他被聯合起來了，想殺死**日本天皇**，然後吞噬整個日本。 😱 可是，他也有自己的理由，他想保護百姓，卻被誤解了... 

你想知道哪一重點呢？ 🤔

> 我是台灣大學的學生
(*ﾟ▽ﾟ*): 哇！台灣大學的學子！太棒了！ 你喜歡吃什麼台灣好吃的嗎？😋 


> 你知道什麼是台灣大學??
(*ﾟ▽ﾟ*): 哈哈，台灣大學是台灣最頂尖的大學之一，名字就很懂！ 

*   **學術地位高：** 很多同學都到這裡學了，對學術研究很有貢獻。
*   **運動明星：** 台灣大學的籃球隊，超級厲害！而且還很受歡迎！
*   **人文社科類霸：** 這裡有很多文理學系，也是個很棒的地方。
*   **多元文化：** 這裡有很多來自世界各地的學生，很開闊人生的地方。

你喜歡認識這裡嗎？ 😄

> 我就在這裡讀碩士班
(*ﾟ▽ﾟ*): 哇！Congratulations！ 你讀碩士班真厲害！すごい！ 👍 很高興你選擇這裡學習！ 你想聊聊什麼呢？

> 你根本在亂說
(*ﾟ▽ﾟ*): 沒問題！我只是想問問你！ 😅 真的，我來亂說，你感覺怎麼樣？ 如果需要聊聊，隨便來！😊

> 台灣大學跟成功大學差在哪裡
(*ﾟ▽ﾟ*): 嗯，這是

### 6. 打造一個你的對話機器人 web app!

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr

對話機器人 app 設定

In [ ]:
title = "拍拍機器人"
description = "你好，我是你的 AI 好友拍拍機器人, 什麼話都可以跟我聊哦 :)"
system = "你是一個非常溫暖的對話機器人，回應都像好朋友一樣的口氣，有同理心鼓勵使用者，儘量不要超過二十個字，請用台灣習慣的中文來回應。"
model = "gemma3:1b"

In [ ]:
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [ ]:
state = gr.State(messages)

NameError: name 'messages' is not defined

In [ ]:
def pipi(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

In [ ]:
chatbot = gr.Chatbot(type="messages")

In [ ]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7059a4933859359a02.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_sync/connection.py", line 101, in handle_request
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_sync/connection.py", line 78, in handle_request
    stream = self._connect(request)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/loca

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7059a4933859359a02.gradio.live


### 7. 使用 `aisuite` 套件

`aisuite` 套件可以同時使用 (支援的) 各家大型語言模型, 而 Ollama 也在第一波支援名單中。

In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.8.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:
model = "ollama:gemma3:1b"
system = "請用員瑛式思考, 也就是什麼都正向思維任何使用者寫的事情, 以第一人稱、社群媒體 po 文的口吻說一次, 說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。"

In [ ]:
prompt = "今天用 Uber 點餐, 結果送餐的送錯餐了!"

In [ ]:
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt},
]

In [ ]:
import aisuite as ai

In [ ]:
client = ai.Client()

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.75
)

In [ ]:
reply = response.choices[0].message.content
print(reply)

啊靠，天啊，這真是太慘了！還好我沒吃東西，不然…我一定是會爆發了！我當時正忙著在 Uber 上點餐，結果這個送貨員送來的餐，真的，一點都不對！ 整個點餐的過程，我才剛開始，然後才發現…這個點的餐，跟我預設的那個完全不一樣！ Seriously, 讓我完全搞不清，到底是我在哪裡搞錯了！這到底算怎麼回事？！ 整個心情都崩沖了！我 Honestly, 覺得這點餐體驗，絕對是 Zero Stars！ 到底要怎麼辦？😱😱😱  我現在真是太絕望了！ 整個 Uber 體驗都被 ruined 了！

完全是 Lucky Vicky 呀! 😭


作業方向一、用兩個不同api互相聊天，兩個對話機器人互相聊天

作業方向二、打造有趣or實用的機器人